In [1]:
# pydantic is a data validation library for python

In [2]:
import os 
import groq

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [3]:
from typing import List
from pydantic import BaseModel, Field

In [4]:
class User:
    def __init__(self, name:str, age:int, email:str):
        self.name = name
        self.age = age
        self.email = email

In [5]:
foo = User(name="Joe", age='me', email="josep@gmail.com")
foo.age

'me'

In [6]:
class pUser(BaseModel):
    name:str
    age:int
    email:str

In [7]:
foo = pUser(name="Joe", age='me', email="josep@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='me', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [8]:
class Class(BaseModel):
    students: List[pUser]

In [9]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [10]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [11]:
from langchain_core.utils.function_calling import convert_to_openai_function


weather_function = convert_to_openai_function(WeatherSearch)
weather_func = {"type": "function",
         "function": weather_function}

weather_func

{'type': 'function',
 'function': {'name': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'parameters': {'type': 'object',
   'properties': {'airport_code': {'type': 'string'}},
   'required': ['airport_code']}}}

In [14]:
from langchain_groq import ChatGroq

model = ChatGroq()
model_with_function = model.bind(tools=[weather_func])
model.invoke("what is the weather in SF today?")

AIMessage(content='I don\'t have real-time capabilities, so I can\'t provide the current weather in San Francisco. However, you can easily find this information by searching "current weather in San Francisco" on a search engine or weather website. This will provide you with up-to-date weather information, including temperature, wind, and humidity.', response_metadata={'token_usage': {'completion_time': 0.110568981, 'completion_tokens': 70, 'prompt_time': 0.007521328, 'prompt_tokens': 18, 'queue_time': None, 'total_time': 0.11809030899999999, 'total_tokens': 88}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-db50c15c-fa50-4cb3-8135-df0de11db1e1-0')

In [15]:
model_with_forced_function = model.bind(tools=[weather_func])
model_with_forced_function.invoke("What is the weather in SF")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fb0e', 'function': {'arguments': '{"airport_code":"SF"}', 'name': 'WeatherSearch'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.144147759, 'completion_tokens': 91, 'prompt_time': 0.365571138, 'prompt_tokens': 1176, 'queue_time': None, 'total_time': 0.509718897, 'total_tokens': 1267}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a3af48a6-b99e-4739-921c-3ab3b022985c-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SF'}, 'id': 'call_fb0e'}])

In [16]:
model_with_forced_function.invoke("Hi")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sy41', 'function': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.146742375, 'completion_tokens': 93, 'prompt_time': 0.387374229, 'prompt_tokens': 1171, 'queue_time': None, 'total_time': 0.534116604, 'total_tokens': 1264}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c02c9040-51d6-4463-ba35-beea64ff2e1a-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'JFK'}, 'id': 'call_sy41'}])

In [17]:
# Function bound to a chain

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

chain = prompt | model_with_function

In [18]:
chain.invoke({"input":"What is the temperature of SF?"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zamb', 'function': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.14612384, 'completion_tokens': 92, 'prompt_time': 1.520386877, 'prompt_tokens': 1210, 'queue_time': None, 'total_time': 1.666510717, 'total_tokens': 1302}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-00d381e3-9976-4146-b936-ee013ab467e5-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': 'call_zamb'}])

In [33]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

artist_function = convert_to_openai_function(ArtistSearch)
artist_func = {"type": "function",
         "function": artist_function}

artist_func

{'type': 'function',
 'function': {'name': 'ArtistSearch',
  'description': 'Call this to get the names of songs by a particular artist',
  'parameters': {'type': 'object',
   'properties': {'artist_name': {'type': 'string'}, 'n': {'type': 'integer'}},
   'required': ['artist_name', 'n']}}}

In [34]:
tools = [
    weather_func,
    artist_func
]

model_with_functions = model.bind(tools = tools)
model_with_function.invoke("What is the weather in SF")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tqyq', 'function': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.146486725, 'completion_tokens': 92, 'prompt_time': 0.396978241, 'prompt_tokens': 1176, 'queue_time': None, 'total_time': 0.543464966, 'total_tokens': 1268}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-66aaa7fd-a6cc-40f4-bee0-b07d8376b992-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': 'call_tqyq'}])

In [35]:
model_with_function.invoke("What are three songs by Taylor Swift")

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "SongSearch",\n\t\t\t"parameters": {\n\t\t\t\t"properties": {\n\t\t\t\t\t"artist": {\n\t\t\t\t\t\t"type": "string"\n\t\t\t\t\t},\n\t\t\t\t\t"number_of_songs": {\n\t\t\t\t\t\t"type": "number",\n\t\t\t\t\t\t"minimum": 3\n\t\t\t\t\t}\n\t\t\t\t},\n\t\t\t\t"required": [\n\t\t\t\t\t"artist",\n\t\t\t\t\t"number_of_songs"\n\t\t\t\t],\n\t\t\t\t"type": "object"\n\t\t\t}\n\t\t}\n\t},\n\t"parameters": {\n\t\t"artist": "Taylor Swift",\n\t\t"number_of_songs": 3\n\t}\n}\n</tool-use>'}}

In [ ]:
model_with_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9hvg', 'function': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.149704238, 'completion_tokens': 93, 'prompt_time': 0.186787804, 'prompt_tokens': 1172, 'queue_time': None, 'total_time': 0.336492042, 'total_tokens': 1265}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b2afd69d-03a2-4019-835b-f0022abc59ec-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'JFK'}, 'id': 'call_9hvg'}])